In [1]:
import numpy as np
import readline
%pylab inline
%load_ext autoreload
%autoreload 2
%precision 3

Populating the interactive namespace from numpy and matplotlib


'%.3f'

In [2]:
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer
lemma = nltk.wordnet.WordNetLemmatizer()

tt = TweetTokenizer(preserve_case=False)

from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
stops.update({"#semst", "...", "rt"})
ps = nltk.stem.porter.PorterStemmer()

def tokenize(t):
    tokens = tt.tokenize(t)
    tokens = [tok for tok in tokens if not ((len(tok) == 1 and not tok.isalpha()) or tok in stops)]
    tokens = [ (ps.stem(tok) if tok[0] != "#" else tok) for tok in tokens]
    return tokens

In [26]:
fp = open("../semeval2016.6/StanceDataset/trump_cleaned_train.txt", "r")
train_tweets = fp.readlines()
train_tweets = [line.strip() for line in train_tweets]
tokens_train = [tokenize(l) for l in train_tweets]
fp.close()

In [27]:
train_tweets[:5]

["Dear #DonaldTrump: Do you want to see #HillaryClinton 's birth certificate too? If so, PLEASE FUCK OFF! #UniteBlue #LibCrib #Hillary2016",
 '#Trump compares #JebBush to #liberal U.S. President Barack #Obama and #Obamacare. #Bush2016 #Trump2016 #Decision2016 https://t.co/Afj91uPyON',
 'Donald Trump blows his trumpet once again, but now he totally blows it. http://t.co/WCWodICfI7 #2016 #HillaryClinton #Trump2016',
 'Why we need #feminism.  #everydaysexism   #DonaldTrump #HilaryClinton2016 http://t.co/fcs2r3pQmu',
 '#DonaldTrump has to be the lowest form of #scumbag out there. The man is vile, the #HillaryClinton remark exposes his #douchebaggery']

In [28]:
# tokens_train[:5]

In [29]:
train = pd.DataFrame(list(zip(train_tweets, tokens_train, map(len, tokens_train))), 
               columns =['Tweet', 'tokens', 'num_tokens']) 
train.head()

,Tweet,tokens,num_tokens
0,Dear #DonaldTrump: Do you want to see #Hillary...,"[dear, #donaldtrump, want, see, #hillaryclinto...",12
1,#Trump compares #JebBush to #liberal U.S. Pres...,"[#trump, compar, #jebbush, #liberal, u, presid...",13
2,"Donald Trump blows his trumpet once again, but...","[donald, trump, blow, trumpet, total, blow, ht...",10
3,Why we need #feminism. #everydaysexism #Don...,"[need, #feminism, #everydaysexism, #donaldtrum...",6
4,#DonaldTrump has to be the lowest form of #scu...,"[#donaldtrump, lowest, form, #scumbag, man, vi...",10


In [30]:
test = pd.read_csv("../semeval2016.6/StanceDataset/B_test.csv", delimiter = "\t")
# del test["Opinion Towards"]
# del test["Sentiment"]
test["tokens"] = [ tokenize(t) for t in test["Tweet"].values ]
test["num_tokens"] = [len(tt) for tt in test["tokens"].values ]
test.head()

,Stance,Tweet,tokens,num_tokens
0,NONE,@2014voteblue @ChrisJZullo blindly supporting ...,"[@2014voteblu, @chrisjzullo, blindli, support,...",10
1,NONE,@ThePimpernelX @Cameron_Gray @CalebHowe Total...,"[@thepimpernelx, @cameron_gray, @calebhow, tot...",12
2,NONE,@JeffYoung @ThePatriot143 I fully support full...,"[@jeffyoung, @thepatriot143, fulli, support, f...",11
3,AGAINST,@ABC Stupid is as stupid does! Showedhis true ...,"[@abc, stupid, stupid, showedhi, true, color, ...",12
4,NONE,@HouseGOP we now have one political party. The...,"[@housegop, one, polit, parti, socialist, part...",9


In [31]:
def search(keyword):
#     subset = train.query('Target==@target')
    filtered = train[train.apply(lambda row: keyword in row["tokens"], axis=1)]
    return filtered["Tweet"]#.reset_index()

In [32]:
from collections import Counter
from itertools import chain, product
all_tokens = Counter(chain(*train["tokens"].values))
# plt.hist(np.log2(list(all_tokens.values()) ), log=True)
all_tokens.most_common(100)

[('#donaldtrump', 34112),
 ('#trump2016', 14536),
 ('trump', 8108),
 ('@realdonaldtrump', 6371),
 ('donald', 4920),
 ('like', 2800),
 ('say', 2410),
 ('get', 2070),
 ('mexican', 2010),
 ('immigr', 2009),
 ('#trump', 1877),
 ('#gop', 1756),
 ('fire', 1752),
 ('make', 1728),
 ('via', 1724),
 ('vote', 1696),
 ('go', 1689),
 ('presid', 1668),
 ('peopl', 1594),
 ('think', 1548),
 ('need', 1495),
 ('us', 1483),
 ('america', 1424),
 ('want', 1387),
 ('#nbc', 1320),
 ('illeg', 1247),
 ('right', 1238),
 ("he'", 1215),
 ('comment', 1212),
 ('one', 1211),
 ('know', 1167),
 ('run', 1157),
 ('nbc', 1151),
 ('time', 1150),
 ('would', 1138),
 ('racist', 1128),
 ('take', 1127),
 ('#makeamericagreatagain', 1123),
 ('u', 1107),
 ('support', 1104),
 ('#tcot', 1087),
 ('good', 1065),
 ("i'm", 1062),
 ('word', 1050),
 ("#donaldtrump's", 1041),
 ('love', 1034),
 ('gop', 1029),
 ('candid', 1028),
 ('talk', 1009),
 ('back', 1003),
 ('give', 957),
 ('poll', 946),
 ('stop', 930),
 ('see', 928),
 ('great', 928),

In [36]:
pd.options.display.max_colwidth = 150
# FAVOR #Trump2016 #makeamericagreatagain
# AGAINST hate racist
print(search("hate").to_string())

118                Difference between #DonaldTrump and #DylannRoof? A handgun. Both hate mankind only, they use different weapons to hate. Proud of #Univision
134                                                           #DonaldTrump, your hateful remarks have consequences. #univision #missusa http://t.co/ZFQ3tNuSh9
184                                                 Today I feel a very strong and specific hate for both homophobes and people who will vote for #DonaldTrump
362                                                  Ricky Martin slams #DonaldTrump over his hateful comments about Mexican immigrants http://t.co/8mSFbWyNPm
450                 #DonaldTrump blame anyone you want, but in reality the Mexican people hate you for being a racist piece of sh.. You're not gonna win this.
507                         #DonaldTrump Stop the hate👊 @somosmitu @wearemitu @realDonaldTrump #imnotacriminal Share it / Compartelo 🙏  http://t.co/gMDmYdlTqT
586                          @RandyFalco Thank

In [37]:
def assign(pos_kws, neg_kws):
    subset = train
    pos_idxs = subset.apply(lambda row: len (set(pos_kws) & set(row["tokens"])) > 0, axis=1).values
    neg_idxs = subset.apply(lambda row: len (set(neg_kws) & set(row["tokens"])) > 0, axis=1).values
    inferred = -1.0 * np.ones(len(subset))
    inferred[pos_idxs] = 1.0
    inferred[neg_idxs] = 0.0
    subset["labels"] = inferred
    return subset

In [39]:
subset = assign(["#freethinker", "#freedom"], ["jesu", "lord", "bless", "#teamjesus"] )
# inferred
# pd.crosstab(index=subset['labels'], columns=train['Stance'], margins = True)

In [40]:
from sklearn.semi_supervised import LabelPropagation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
bow = TfidfVectorizer(
            tokenizer = lambda x: x, ## we already have list of entities
            lowercase = False,
            binary = False,
            use_idf = True,
            preprocessor = None,
            norm = None,
            min_df = 2,
        )
X = bow.fit_transform(subset["tokens"])
model = LabelPropagation(kernel = 'rbf', gamma = 0.1)
model.fit(X.todense(), subset["labels"].values)
subset["predict"] = model.transduction_
pd.crosstab(index=subset['predict'], columns=subset['Stance'], margins = True)

In [ ]:
subset_test = test
X_test = bow.transform(subset_test["tokens"])
subset_test['predict'] = model.predict(X_test)
pd.crosstab(index=subset_test['predict'], columns=subset_test['Stance'], margins = True)